In [3]:
file_id = '19bnYs80DwuUimHM65UV3sylsCn2j1vziPOwzBwQrebw'

url = f'https://docs.google.com/document/d/{file_id}/export?format=docx'

In [4]:
import requests
import io
import docx

In [12]:
response = requests.get(url)
response.raise_for_status

with io.BytesIO(response.content) as f_in:
    doc = docx.Document(f_in)

In [13]:
doc

```We can take paragraphs via lines from our docs file```

In [15]:
doc.paragraphs[0]

```Convert it to text format```

In [16]:
p = doc.paragraphs[0]

p.text

'The purpose of this document is to capture frequently asked technical questions'

In [17]:
p = doc.paragraphs[5]

p.text

'Join the course Telegram channel with announcements.'

```Also we can find out which type of line we get by check it with commands style. For example we have several types of lines: normal (casual text), Heading 2 (sub section), Heading (section name). By defining type of text we can parse it to format: 'course_name' - 'question' - 'answer'```

In [33]:
p = doc.paragraphs[1]

p.text, p.style

('General course-related questions',
 _ParagraphStyle('Heading 1') id: 138942281587520)

In [25]:
p.style.name.lower()

'heading 2'

```set text type style```

In [26]:
question_heading_style = 'heading 2'
section_heading_style = 'heading 1'

In [32]:
documents = []

current_answer = []
current_section = ''
current_question = ''


for p in doc.paragraphs:
    p_text = p.text.strip() # in order to delete whitespaces
    p_stye = p.style.name.lower()

    if p_text == '':
        continue

    if p_style == section_heading_style:
        current_section = p.text
        current_answer = []
        continue 

    if p_style == question_heading_style:
        if current_section and current_question and current_answer:
            answer = '\n'.join(current_answer)
    
            documents.append({
                'section': current_section,
                'question': current_question,
                'text': answer
            })
    
            current_question = p_text
        continue

    current_answer.append(p_text)


if current_answer:
    answer = '\n'.join(current_answer)

    documents.append({
        'section': current_section,
        'question': current_question,
        'text': answer
    })

SyntaxError: invalid syntax (413049186.py, line 33)

In [27]:
doc

In [28]:
p

In [29]:
p.text

'Course - When will the course start？'

In [30]:
p.style

_ParagraphStyle('Heading 2') id: 138942282203296

In [31]:
p.style.name

'Heading 2'